In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import string

In [40]:
PATH = '/content/water_problem_nlp_en_for_Kaggle_100.csv'
dataset = pd.read_csv(PATH, sep=";")
dataset.head(10)

,text,env_problems,pollution,treatment,climate,biomonitoring
0,In the Southern Bug basin wastewater is discha...,1,1.0,NaN,NaN,NaN
1,This amount is distributed as follows: urban a...,1,1.0,NaN,NaN,NaN
2,Despite the similar volumes of discharged wast...,1,1.0,NaN,NaN,NaN
3,They bring 84% of organic pollution 86% of pho...,1,1.0,NaN,NaN,NaN
4,The input of the industry is between 7–21% and...,1,1.0,NaN,NaN,NaN
5,Settlements that do not have sewage systems us...,1,NaN,1.0,NaN,NaN
6,Equipment of WWTPs is very worn-out the amount...,1,NaN,1.0,NaN,NaN
7,The current state of sewage networks is a sour...,1,NaN,1.0,NaN,NaN
8,Consequently in Ukraine on average 2 accidents...,1,1.0,NaN,NaN,NaN
9,The load of pollutants from accidental dischar...,0,NaN,NaN,NaN,NaN


In [41]:
# Lets replace NaN with 0, as its multilabel problem we need to format it properly
dataset.fillna(value=0, inplace=True)

In [42]:
list(dataset.iloc[12, 1:].to_numpy().astype(np.float32))

[1.0, 1.0, 1.0, 0.0, 0.0]

In [43]:
def clean_string(dataframe, field):
  dataframe[field] = dataframe[field].str.replace("[{}]".format(string.punctuation), '')
  dataframe[field] = dataframe[field].str.lower()

  lines = []
  target = []

  for idx, text in enumerate(dataframe[field].to_numpy()):

    if text != '':
      lines.append(text.strip())
      target.append(list(dataset.iloc[idx, 1:].to_numpy().astype(np.float32)))
  
  return lines, target

In [44]:
dataset, target = clean_string(dataset, 'text')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [45]:
dataset[:10]

['in the southern bug basin wastewater is discharged from 341 organized sources map 5 in 2011 reference year 211 mln m3 was discharged in total',
 'this amount is distributed as follows urban and rural housing wastewater treatment facilities wwtps  vodocanals – 70 mln m3 57 users industry – 50 mln m3 79 users agriculture – 90 mln m3 193 users other users – 1 mln m3 12 users picture 21',
 'despite the similar volumes of discharged wastewater major part of pollutants comes with communal wwtps',
 'they bring 84 of organic pollution 86 of phosphate ions and 84 of mineral nitrogen 91 of ammonia nitrogen 87 nitrate nitrogen and 79 nitrite nitrogen',
 'the input of the industry is between 7–21 and agriculture has the lowest impact on water bodies  0–6 of the 92 urban areas only 51 localities 55 have centralized collection of communal waste waters and their monitoring among the 2878 villages 6 of them 02 have such a monitoring',
 'settlements that do not have sewage systems usually do not have

In [46]:
target[:10]

[[1.0, 1.0, 0.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 1.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0]]

In [47]:
len(dataset) == len(target)

True

In [48]:
def train_val_split(data, target, train_size=0.8):
  train_len = int(len(data) * train_size)
  index = tf.random.shuffle(tf.range(len(data)))
  x_train = tf.gather(data, index[:train_len])
  y_train = tf.gather(target, index[:train_len])

  x_val = tf.gather(data, index[train_len:])
  y_val = tf.gather(target, index[train_len:])

  return (x_train, y_train), (x_val, y_val)

In [49]:
train, val = train_val_split(dataset, target)

In [50]:
train = tf.data.Dataset.from_tensor_slices(train)
val = tf.data.Dataset.from_tensor_slices(val)

In [51]:
for X, Y in train.take(1):
  print(X)
  print(Y)

tf.Tensor(b'this method allows extracting only a part of pollutants containing nitrogen and phosphorus', shape=(), dtype=string)
tf.Tensor([1. 1. 1. 0. 0.], shape=(5,), dtype=float32)


In [52]:
MAX_SEQUENCE = 250
VOCABSIZE = 10000

vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCABSIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE
)

In [53]:
text = train.map(lambda text, label: text)
vectorizer.adapt(text)

In [54]:
def vectorize_text(text, labels=None):
  if labels is not None:
    return vectorizer(text), labels
  return vectorizer(text)

In [55]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train = train.map(vectorize_text, num_parallel_calls=AUTOTUNE)
val = val.map(vectorize_text, num_parallel_calls=AUTOTUNE)

In [56]:
for X, Y in train.take(1):
  print(X.shape)
  print(Y.shape)
  print(X)
  print(Y)

(250,)
(5,)
tf.Tensor(
[ 43 400 605 484 191  32   9   3  56 543  13   5  44   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0

In [57]:
BATCH_SIZE = 64
train = train.cache().shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val = val.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [58]:
for line, label in train.take(1):
    print(line.shape)
    print(label.shape)

(64, 250)
(64, 5)


In [71]:
embedding = 64
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(input_dim=VOCABSIZE,
                                                              output_dim=embedding,
                                                              input_length=MAX_SEQUENCE,
                                                              mask_zero=True),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                                    tf.keras.layers.Dense(64, activation='relu'),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(5)
])

In [72]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 250, 64)           640000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 250, 128)         66048     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                

In [73]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.BinaryAccuracy()])

In [74]:
earlystopping = tf.keras.callbacks.EarlyStopping(patience=20)
checkpoint = tf.keras.callbacks.ModelCheckpoint("multilabel_nlp.h5", save_only_best=True, monitor="val_loss")
#scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch : learning_rate * 0.1 ** (epoch/20) )

In [75]:
history = model.fit(train, validation_data = val, epochs=100, callbacks=[earlystopping, checkpoint])

Epoch 1/100
2/2 [==============================] - 28s 8s/step - loss: 0.6928 - binary_accuracy: 0.7425 - val_loss: 0.6885 - val_binary_accuracy: 0.6900
Epoch 2/100
2/2 [==============================] - 2s 958ms/step - loss: 0.6869 - binary_accuracy: 0.7425 - val_loss: 0.6812 - val_binary_accuracy: 0.6900
Epoch 3/100
2/2 [==============================] - 2s 989ms/step - loss: 0.6770 - binary_accuracy: 0.7425 - val_loss: 0.6711 - val_binary_accuracy: 0.6900
Epoch 4/100
2/2 [==============================] - 2s 913ms/step - loss: 0.6658 - binary_accuracy: 0.7425 - val_loss: 0.6564 - val_binary_accuracy: 0.6900
Epoch 5/100
2/2 [==============================] - 2s 980ms/step - loss: 0.6473 - binary_accuracy: 0.7425 - val_loss: 0.6344 - val_binary_accuracy: 0.6900
Epoch 6/100
2/2 [==============================] - 2s 927ms/step - loss: 0.6260 - binary_accuracy: 0.7450 - val_loss: 0.6036 - val_binary_accuracy: 0.6900
Epoch 7/100
2/2 [==============================] - 2s 900ms/step - loss: